In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import miceforest as mf
import re

sns.set()

In [2]:
response_vars_df1 = pd.read_csv("data/full_imputed_response_df_iter0.csv")
response_vars_df1.shape

(24471, 226)

In [3]:
predictor_vars_df1 = (pd.read_csv("data/full_imputed_predictor_df_iter0.csv")).iloc[:,1:]
cleaned_cols = [''.join(re.split(r'(\d+)', col)[2:]) for col in predictor_vars_df1.columns]
predictor_vars_df1.columns = cleaned_cols
predictor_vars_df1.shape

(24471, 401)

In [4]:
def split_cnum(cnum_var):
    cnum_var = cnum_var[1:]
    nums = cnum_var.rstrip('0123456789')
    var_name = cnum_var[len(nums):]
    return var_name

def create_composite_vars(full_df,predictors=True):
    if predictors: 
        trs = "W2"
    else:
        trs = ""
    
    de_var_names = [col for col in full_df.columns if "DE" in col]
    full_df["DE_sum"] = full_df.loc[:,de_var_names].sum(axis=1)

    binary_positive_vars = [col for col in [(str(col).upper())+"_DUMMY1" for col in ("dncl10w2,dncl11w2,dncl12w2,pncl1w2,pncl2w2,"+\
                            "pncl7w2,pncl12w2,pncl13w2,cscaw2,clhcw2,flihcw2,tomenw2,"+\
                            "tomepgw2,tomelnw2,tomesuw2,tomehdw2,dncl9w2,dncl10w2,dncl11w2,"+\
                            "dncl12w2,cbnpw2".replace("w2",trs)).split(",")] if col in full_df.columns]
    binary_negative_vars = [col for col in [(str(col).upper())+"_DUMMY1" for col in ("dncl1w2,dncl2w2,dncl3w2,dncl4w2,dncl5w2,dncl6w2,"+\
                            "dncl7w2,dncl8w2,dncl14w2,pncl3w2,pncl4w2,pncl5w2,pncl6w2,cilw2,cflw2,"+\
                            "cmosw2,cbiw2,cgiw2,tomeew2,tomempw2,tomergw2,"+\
                            "tomerbw2,tomemfw2,tomepmw2,tomethpw2,tomerew2,tomesgw2"\
                                                                     .replace("w2",trs)).split(",")] if col in full_df.columns]
    cat3_positive_vars = [col for col in [col+str(i) for col in [(str(col).upper())+"_DUMMY" for col in \
                          ("infdw2,infcw2".replace("w2",trs))\
                                                 .split(",")] for i in range(1,3)] if col in full_df.columns]
    cat4_positive_vars = [col for col in [col+str(i) for col in [(str(col).upper())+"_DUMMY" for col in \
                          ("dn9w2,dn10w2,dn11w2,dn12w2".replace("w2",trs))\
                                                 .split(",")] for i in range(1,4)] if col in full_df.columns]
    cat4_negative_vars = [col for col in [col+str(i) for col in [(str(col).upper())+"_DUMMY" for col in \
                          ("dn14w2,dn1w2,dn2w2,dn3w2,dn4w2,dn5w2,dn6w2,dn7w2,dn8w2".replace("w2",trs))\
                                                 .split(",")]\
                           for i in range(1,4)] if col in full_df.columns]
    cat5_positive_vars = [col for col in [col+str(i) for col in [(str(col).upper())+"_DUMMY" for col in \
                          ("pn1w2,pn2w2,pn10w2,pn11w2,pn12w2,pn13w2".replace("w2",trs)).split(",")]\
                          for i in range(1,5)] if col in full_df.columns]
    cat5_negative_vars = [col for col in [col+str(i) for col in [(str(col).upper())+"_DUMMY" for col in \
                          ("pn3w2,pn4w2,pn5w2,pn6w2,pn7w2,pn9w2".replace("w2",trs)).split(",")]\
                           for i in range(1,5)] if col in full_df.columns]
    #cat3_negative_vars: none
    
    

    full_df["binary_vars"] = full_df.loc[:,binary_positive_vars]\
    .sum(axis=1) - full_df.loc[:,binary_negative_vars].sum(axis=1)

    full_df["composite_conflict_score"] = full_df["binary_vars"] - \
    (full_df["DE_sum"] * 0.25)

    return full_df

# TODO: Add 2< cat variables

In [5]:
predictor_comp_df1 = create_composite_vars(predictor_vars_df1)
response_comp_df1 = create_composite_vars(response_vars_df1)

complete_vars_df1 = pd.concat([predictor_comp_df1,response_comp_df1],axis=1)
complete_vars_df1.to_csv("data/preprocessed_conflict_data_iter0MICE.csv")
complete_vars_df1.shape

(24471, 633)

## Below: Naively Imputed Data & Old EDA

Used for project update.

In [6]:
xyz

NameError: name 'xyz' is not defined

In [ ]:
original_df = pd.read_csv("data/anticonflict_study_dataframe.csv",low_memory=False)
#original_indices = pd.read_csv("data/tenth_sample_indices.csv")['0']
original_sample = original_df #.iloc[original_indices.values,:]
original_sample.loc[:,[col for col in original_sample.columns if col.startswith("RTSM")]] =\
original_sample.loc[:,[col for col in original_sample.columns if col.startswith("RTSM")]].fillna(0)

original_sample = original_sample.iloc[:,1:]

response_comp_df1 = create_composite_vars(response_vars_df1)

In [ ]:
response_comp_df1["binary_vars"].hist()
plt.show()

In [ ]:
response_comp_df1["DE_sum"].describe()

In [ ]:
response_comp_df1["DE_sum"].hist(bins=50)
plt.show()

In [ ]:
response_comp_df1["composite_conflict_score"].hist(bins=50)
plt.show()

In [ ]:
#(response_comp_df1["composite_conflict_score"]).to_csv("data/imputed_response_composite_iter1.csv")

In [ ]:
response_comp_df1["composite_conflict_score"].isna().value_counts()

In [ ]:
school_grouped_predictors = pd.get_dummies(original_sample[["SCHID","SCHTREAT","AGEC_NEW","GENDER",
                                "GR","COLL","CELL","FSCH","RTSM1","RTSM2","RTSM3","RTSM4","RTSM5",
                                                           "RTSM6","RTSM7","RTSM8","RTSM9"]],
               drop_first=True).groupby("SCHID",as_index=False).mean()
response_comp_df1["SCHID"] = original_sample["SCHID"]
school_grouped_response = pd.DataFrame(response_comp_df1.groupby("SCHID",as_index=False)["composite_conflict_score"].mean())

tenth_initial_df = school_grouped_predictors.merge(school_grouped_response)
tenth_initial_df.to_csv("data/noncompliance_school_level_data.csv")
#tenth_initial_df.corr()["SCHTREAT_(1) Treatment school (Roots meetings 2012-2013)"]

In [ ]:
student_lvl_predictors = pd.get_dummies(original_sample[["SCHID","SCHTREAT","AGEC_NEW",
                                                         "GENDER","GR","COLL","CELL","FSCH","TREAT",
                                                         "RTSM1","RTSM2","RTSM3","RTSM4","RTSM5",
                                                           "RTSM6","RTSM7","RTSM8","RTSM9"]],
                                       drop_first=True)
student_lvl_predictors["Composite_Conflict_Score"] = response_comp_df1["composite_conflict_score"]
student_lvl_predictors.to_csv("data/noncompliance_student_level_data.csv")
#